In [1]:
import yaml
from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler
from paprika.evaluator.utils import get_benchmarks

## Select host-guest pairs

In [2]:
host_guest = {
    "bcd": ["hex"],
    "cb7": ["mvn"],
    "oam": ["cbn"],
}

## Store charges from SDF files to `LibraryChargeHandler`

In [3]:
guest_list = []
library_charge_handler = LibraryChargeHandler(version=0.3)
taproom = get_benchmarks()

for host in host_guest:
    # Get Charges for Host molecules
    metadata = taproom["host_guest_systems"][host]

    with open(metadata["yaml"]["p"], "r") as f:
        host_metadata = yaml.load(f, Loader=yaml.FullLoader)

    host_mol = Molecule.from_file(
        f"{metadata['path']}/{host_metadata['structure']['sdf']}",
        allow_undefined_stereo=True,
    )
    host_charges = host_mol.partial_charges
    host_smirks = host_mol.to_smiles(
        isomeric=True, explicit_hydrogens=True, mapped=True
    )

    library_charge_handler.add_parameter(
        parameter_kwargs={
            "smirks": host_smirks,
            **{f"charge{i + 1}": charge for i, charge in enumerate(host_charges)},
        }
    )

    # Get Charges for Guest molecules
    for guest in host_guest[host]:
        with open(metadata[guest]["yaml"], "r") as f:
            guest_metadata = yaml.load(f, Loader=yaml.FullLoader)

        guest_mol = Molecule.from_file(
            f"{metadata[guest]['path']}/{guest_metadata['structure']['sdf']}",
            allow_undefined_stereo=True,
        )
        guest_charges = guest_mol.partial_charges
        guest_smirks = guest_mol.to_smiles(
            isomeric=True, explicit_hydrogens=True, mapped=True
        )

        if guest_smirks in guest_list:
            continue

        guest_list.append(guest_smirks)

        library_charge_handler.add_parameter(
            parameter_kwargs={
                "smirks": guest_smirks,
                **{f"charge{i + 1}": charge for i, charge in enumerate(guest_charges)},
            }
        )

## Write Charges to File

In [4]:
ff = ForceField()

ff.register_parameter_handler(library_charge_handler)

ff.to_file("taproom_AM1BCC.offxml")